This notebook contains the details about the processing part, done partly by internal pipeline, which couldn't be fully published, therefore described in words. 

# Merging the data by chromosomes

In [ ]:
# Go to the directory with normalized data from previous step (3_normalize_data.ipynb)
cd .../450k_data/

In [ ]:
# Merging all the norm.part* files without headers
# Output stored in anther directory .../450k/RAP_output_per_chr/
first=1
for i in {1..22}; do for f in ./chr$i/output.chr$i.part*.norm.csv.gz; do if [ "$first" ]; then zcat "$f"; first=; else zcat "$f"| tail -n +2; fi; done > .../450k/RAP_output_per_chr/chr$i.all_parts.csv ; done

# Update samples

The file `.../450k/samples/ancestry/european_non_related_to_include.txt` is a combination of the output from script `pipeline/1_generate_unrelaed_samples.ipynb` (`./450k/samples/relatedness/related_samples_to_remove_final.txt`) and samples with non-european genetic ancestry (based on data-field `22006`).


The file `.../450k/samples/withdrawal_samples.txt` contains samples that need to be withdrew due to revoked consent. 

In [1]:
#create an updated list of identifiers (rempve most updated withdrawals)
fgrep -vf /.../450k/samples/withdrawal_samples.txt .../450k/samples/ancestry/european_non_related_to_include.txt > .../450k/samples/european_non_related_no_withdrawal_to_include.txt

In [ ]:
#create updated per-chr files with samples to include
for i in {1..22}; do awk -v FS="," 'FNR==NR{arr[$0];next} ($6 in arr)' .../450k/samples/european_non_related_no_withdrawal_to_include.txt .../450k/RAP_output_per_chr/chr$i.all_parts.csv> .../450k/RAP_output_per_chr/chr$i.all_parts_updated_samples.csv; done

In [ ]:
#Extracting the header randomly from one of the files:
zcat ./chr22/output.chr22.part0.norm.csv.gz|grep chrom > .../450k/RAP_output_per_chr/header.txt

In [ ]:
#Adding header to the individual merged files per chromosome:
for i in {1..22}; do cat .../450k/RAP_output_per_chr/header.txt  .../450k/RAP_output_per_chr/chr$i.all_parts_updated_samples.csv > .../450k/RAP_output_per_chr/chr$i.all_parts_final.csv; done

In [ ]:
#Removing un-needed files:
rm -f .../450k/RAP_output_per_chr/*all_parts.csv
rm -f .../450k/RAP_output_per_chr/*all_parts_updated_samples.csv

# Annotation

Annotation runs for each chromosome separately, adds necessary information for PLPs selection.

# PLPs selection

PLPs selection runs for each chromosome separately using `plp_selection_scripts/select_plps_pandas.py`, as described in the paper. 

Saves them as `.../450k/plp_selection/basic/new_gene_names/new_freq/all_chr_total_presumable_plps_HFE_final_sorted.txt`.